In [2]:
def cmplx_fmt(z):
    if z.imag == 0.0:
        return f"{z.real:+g}"
    elif z.real == 0.0:
        return f"{z.imag:+g}i"
    else:
        return f"({z.real:+g}{z.imag:+g}i)"
class c:
    '''c operator (fermionic)'''
    spin = "u"
    label = ""
    dag = False
    def __init__(self, spin, label, dag=False):
        self.spin = str(spin)
        self.label = str(label)
        self.dag = bool(dag)
    def __lt__(self, other):
        return self.dag < other.dag or self.spin > other.spin or self.label < other.label
    def __eq__(self, other):
        return self.dag == other.dag and self.spin == other.spin and self.label == other.label
    def __str__(self):
        str_spin = r"\uparrow" if self.spin == "u" else r"\downarrow" if self.spin == "d" else self.spin
        str_dag = r"^{\dagger}" if self.dag else ""
        return f"c_{{{str_spin} {self.label}}}{str_dag}"
    def __add__(self, other):
        return exp(self) + exp(other)
    def __radd__(self, other):
        return exp(other) + exp(self)
    def __sub__(self, other):
        return exp(self) - exp(other)
    def __rsub__(self, other):
        return exp(other) - exp(self)
    def __mul__(self, other):
        if type(other) is cp:
            return cp(self) * other
        if type(other) is c:
            return cp(self) * cp(other)
        try:
            return exp(self) * exp(other)
        except NotImplementedError:
            return NotImplemented
    def __rmul__(self, other):
        return exp(other) * exp(self)
class cp:
    '''product of c operators'''
    cs = []
    def __init__(self, x):
        if type(x) is c:
            self.cs = [x]
        elif type(x) is cp:
            self.cs = x.cs
        elif type(x) is list:
            self.cs = x
    def __lt__(self, other):
        return self.cs < other.cs
    def __eq__(self, other):
        return self.cs == other.cs
    def __str__(self):
        return " ".join([str(o) for o in self.cs])
    def __add__(self, other):
        return exp(self) + exp(other)
    def __radd__(self, other):
        return exp(other) + exp(self)
    def __sub__(self, other):
        return exp(self) - exp(other)
    def __rsub__(self, other):
        return exp(other) - exp(self)
    def __mul__(self, other):
        if type(other) is cp:
            return cp(self.cs + other.cs)
        if type(other) is c:
            return self * cp(other)
        try:
            return exp(self) * exp(other)
        except NotImplementedError:
            return NotImplemented
    def __rmul__(self, other):
        return exp(other) * exp(self)
    def _wick(self):
        w = exp([])
        for i in range(1, len(self.cs)):
            if self.cs[0].spin == self.cs[i].spin and self.cs[0].dag != self.cs[i].dag:
                p = 1.0 if i % 2 == 1 else -1.0
                if len(self.cs) == 2:
                    w = w + p*self.cs[0]*self.cs[i]
                else:
                    w = w + p*self.cs[0]*self.cs[i]*cp(self.cs[1:i] + self.cs[i+1:])._wick()
        return w
    def wickstr(self):
        def grnstr(a, b):
            spnstr = r"\uparrow" if a.spin == "u" else r"\downarrow" if a.spin == "d" else a.spin
            if (not a.dag) and b.dag:
                return f"g_{{{a.label} {b.label}}}^{{{spnstr}}}"
            elif a.dag and (not b.dag):
                deltastr = fr"\delta_{{{a.label} {b.label}}}" if a.label < b.label \
                           else fr"\delta_{{{b.label} {a.label}}}" if b.label < a.label \
                           else "1"
                return f"({deltastr} - g_{{{b.label} {a.label}}}^{{{spnstr}}})"
        w = self._wick().simplify()
        return "\n".join([("+" if x == 1.0 else "-" if x == -1.0 else cmplx_fmt(x)) + " ".join([grnstr(a, b) for a, b in zip(o.cs[::2], o.cs[1::2])]) for x, o in w.terms])
    def wickcode(self):
        def grnstr(a, b):
            if (not a.dag) and b.dag:
                return f"g{a.spin}{a.label}{b.label}"
            elif a.dag and (not b.dag):
                return f"h{a.spin}{b.label}{a.label}"
            else:
                raise RuntimeError
        w = self._wick().simplify()
        return "".join([("+" if x == 1.0 else "-" if x == -1.0 else cmplx_fmt(x)) + "*".join([grnstr(a, b) for a, b in zip(o.cs[::2], o.cs[1::2])]) for x, o in w.terms])
class exp:
    '''expression'''
    terms = []
    def __init__(self, x):
        if type(x) in (int, float, complex):
            self.terms = [[x, cp([])]]
        elif type(x) in (c, cp):
            self.terms = [[1.0, cp(x)]]
        elif type(x) is list:
            self.terms = x
        elif type(x) is exp:
            self.terms = x.terms
        else:
            print("wtf", x, type(x))
            raise NotImplementedError
    def __str__(self):
        return "\\\\\n".join([f"{cmplx_fmt(x)} {o}" for x, o in self.terms])
    def __add__(self, other):
        if type(other) is exp:
            return exp(self.terms + other.terms)
        else:
            return self + exp(other)
    def __radd__(self, other):
        return exp(other) + self
    def __sub__(self, other):
        return self + -1.0*other
    def __rsub__(self, other):
        return other + -1.0*self
    def __mul__(self, other):
        if type(other) is exp:
            return exp([[x[0]*y[0], x[1]*y[1]] for x in self.terms for y in other.terms])
        try:
            return self * exp(other)
        except NotImplementedError:
            return NotImplemented
    def __rmul__(self, other):
        return exp(other) * self
    def simplify(self):
        if len(self.terms) == 0:
            return exp([])
        sort_terms = sorted(self.terms, key=lambda t: t[1])
        simp_terms = [sort_terms[0][:]]
        for t in sort_terms[1:]:
            if simp_terms[-1][1] == t[1]:
                simp_terms[-1][0] += t[0]
            else:
                simp_terms.append(t[:])
        return exp([t for t in simp_terms if t[0] != 0.0])
    def wickstr(self):
        wicks = [(x, o.wickstr()) for x, o in self.terms]
        return "\\\\\n".join([f"{cmplx_fmt(x)} (\n{w}\n)" for x, w in wicks if w != ""])
    def wickcode(self):
        wicks = [(x, o.wickcode()) for x, o in self.terms]
        return "\n".join([f"{cmplx_fmt(x)}*({w})" for x, w in wicks if w != ""])
class vec:
    '''3-vector'''
    x, y, z = 0.0, 0.0, 0.0
    def __init__(self, x, y, z):
        self.x, self.y, self.z = x, y, z
    def __str__(self):
        return f"({self.x})\hat{{x}}\n+({self.y})\hat{{y}}\n+({self.z})\hat{{z}}"
    def __add__(self, other):
        if type(other) is vec:
            return vec(self.x + other.x, self.y + other.y, self.z + other.z)
        return NotImplemented
    def __sub__(self, other):
        if type(other) is vec:
            return vec(self.x - other.x, self.y - other.y, self.z - other.z)
        return NotImplemented
    def __mul__(self, other):
        if type(other) is vec:
            raise TypeError
        return vec(self.x * other, self.y * other, self.z * other)
    def __rmul__(self, other):
        if type(other) is vec:
            raise TypeError
        return vec(other * self.x, other * self.y, other * self.z)
    def dot(self, other):
        return (self.x * other.x) + (self.y * other.y) + (self.z * other.z)
    def cross(self, other):
        return vec(self.y * other.z - self.z * other.y,
                   self.z * other.x - self.x * other.z,
                   self.x * other.y - self.y * other.x)

In [3]:
def spin(label):
    cud = c("u", label, 1)
    cdd = c("d", label, 1)
    cu = c("u", label)
    cd = c("d", label)
    return vec(cud*cd + cdd*cu, -1j*cud*cd + 1j*cdd*cu, cud*cu - cdd*cd)

def den(label):
    cud = c("u", label, 1)
    cdd = c("d", label, 1)
    cu = c("u", label)
    cd = c("d", label)
    return cud*cu + cdd*cd

def hden(label):
    cud = c("u", label, 1)
    cdd = c("d", label, 1)
    cu = c("u", label)
    cd = c("d", label)
    return cu*cud + cd*cdd

def hole(label):
    cud = c("u", label, 1)
    cdd = c("d", label, 1)
    cu = c("u", label)
    cd = c("d", label)
    return cu*cud*cd*cdd

In [4]:
ss = (hden("h") * spin("i").z * spin("j").z).simplify()
# ss = (c("u", "h", 1) * c("u", "h") * spin("i").x * spin("j").x).simplify()
# ss = (1*c("u", "h", 1) * c("u", "h") * c("u", "i", 1) * c("d", "i") * c("d", "i", 1) * c("u", "j")).simplify()
print(ss.wickcode())

+1*(+guhh*huii*hujj+guhh*huji*guij-guhi*huih*hujj-guhi*hujh*guij+guhj*huih*huji-guhj*hujh*huii)
-1*(+guhh*huii*hdjj-guhi*huih*hdjj)
-1*(+guhh*hdii*hujj-guhj*hujh*hdii)
+1*(+guhh*hdii*hdjj+guhh*hdji*gdij)
+1*(+gdhh*huii*hujj+gdhh*huji*guij)
-1*(+gdhh*huii*hdjj-gdhj*hdjh*huii)
-1*(+gdhh*hdii*hujj-gdhi*hdih*hujj)
+1*(+gdhh*hdii*hdjj+gdhh*hdji*gdij-gdhi*hdih*hdjj-gdhi*hdjh*gdij+gdhj*hdih*hdji-gdhj*hdjh*hdii)


In [3]:
#zz
# if False:
ss = (spin("i").y * spin("j").y).simplify()
print(ss.wickcode())
print("---")
ss = (den("i") * den("j")).simplify()
print(ss.wickcode())
print("---")
# test = c("u", "1", 1)*c("u", "2")*c("u", "3", 1)*c("u", "4")*c("u", "5", 1)*c("u", "6")
# print(test.wickstr())

+1*(+huji*gdij)
+1*(+hdji*guij)
---
+1*(+huii*hujj+huji*guij)
+1*(+huii*hdjj)
+1*(+hdii*hujj)
+1*(+hdii*hdjj+hdji*gdij)
---


In [3]:
#eq10
se = spin("e")
sj = spin("j")
sk = spin("k")
sss = se.dot(sj.cross(sk)).simplify()
eq10 = (-1j*c("u", "d")*c("u", "i", 1)*sss).simplify()
#print(eq10)
#print("---")
#print(eq10.wickstr())
print(eq10.wickcode())

+2*(-gude*huei*hukj*gdjk-gude*huki*guej*gdjk+gudi*huee*hukj*gdjk+gudi*huke*guej*gdjk+gudj*huei*huke*gdjk-gudj*huki*huee*gdjk)
-2*(-gude*huei*hdkj*gujk+gude*huji*guek*hdkj+gudi*huee*hdkj*gujk-gudi*huje*guek*hdkj-gudk*huei*huje*hdkj+gudk*huji*huee*hdkj)
-2*(+gude*huji*gdek*hukj-gude*huki*gdek*hujj-gudi*huje*gdek*hukj+gudi*huke*gdek*hujj-gudj*huji*huke*gdek+gudj*huki*huje*gdek)
+2*(-gude*huji*gdej*hukk-gude*huki*gdej*gujk+gudi*huje*gdej*hukk+gudi*huke*gdej*gujk+gudk*huji*huke*gdej-gudk*huki*huje*gdej)
-2*(-gude*huji*gdej*hdkk+gude*huji*gdek*hdkj+gudi*huje*gdej*hdkk-gudi*huje*gdek*hdkj)
+2*(-gude*huki*gdej*gdjk-gude*huki*gdek*hdjj+gudi*huke*gdej*gdjk+gudi*huke*gdek*hdjj)
+2*(+gudi*hdke*guej*gujk+gudi*hdke*guek*hujj+gudj*huei*hdke*gujk-gudj*huji*hdke*guek+gudk*huei*hdke*hujj+gudk*huji*hdke*guej)
-2*(+gudi*hdje*guej*hukk-gudi*hdje*guek*hukj+gudj*huei*hdje*hukk+gudj*huki*hdje*guek-gudk*huei*hdje*hukj-gudk*huki*hdje*guej)
+2*(+gudi*hdje*guej*hdkk+gudi*hdke*guej*gdjk+gudj*huei*hdje*hdkk+gudj*hu

In [4]:
#eq4
si = spin("i")
sj = spin("j")
sk = spin("k")
ssss = ((si.dot(sj))*(si.dot(sk))).simplify()
eq4 = (c("u", "h")*c("u", "h", 1)*ssss).simplify()
print(eq4.wickcode())

+1*(+guhh*huii*guii*hujj*hukk+guhh*huii*guii*hukj*gujk+guhh*huii*guij*guji*hukk-guhh*huii*guij*gujk*huki-guhh*huii*guik*hujj*huki-guhh*huii*guik*hukj*guji+guhh*huii*huij*guji*hukk-guhh*huii*huij*gujk*huki+guhh*huii*hujj*huii*hukk+guhh*huii*hujj*huki*guik+guhh*huii*hukj*guji*guik+guhh*huii*hukj*gujk*huii-guhh*huji*guii*huij*hukk-guhh*huji*guii*hukj*guik+guhh*huji*guij*huii*hukk+guhh*huji*guij*huki*guik+guhh*huji*guik*huij*huki-guhh*huji*guik*hukj*huii-guhh*huki*guii*huij*gujk+guhh*huki*guii*hujj*guik+guhh*huki*guij*guji*guik+guhh*huki*guij*gujk*huii+guhh*huki*guik*huij*guji+guhh*huki*guik*hujj*huii-guhi*huih*guii*hujj*hukk-guhi*huih*guii*hukj*gujk-guhi*huih*guij*guji*hukk+guhi*huih*guij*gujk*huki+guhi*huih*guik*hujj*huki+guhi*huih*guik*hukj*guji-guhi*huih*huji*guij*hukk+guhi*huih*huji*guik*hukj-guhi*huih*huji*huij*hukk-guhi*huih*huji*hukj*guik-guhi*huih*huki*guij*gujk-guhi*huih*huki*guik*hujj-guhi*huih*huki*huij*gujk+guhi*huih*huki*hujj*guik-guhi*huih*huij*guji*hukk+guhi*huih*huij*gujk*